Install Required Libraries

In [ ]:
!pip install beyondllm youtube_transcript_api llama-index-readers-youtube-transcript llama_index.embeddings.huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 563.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

Set Environment Variables

In [ ]:
from getpass import getpass
import os

hf_token = getpass('Enter Your HuggingfaceHub Token')
google_api_key = getpass('Enter Your Google API Key')

os.environ['HF_TOKEN'] = hf_token
os.environ['GOOGLE_API_KEY'] = google_api_key

Enter Your HuggingfaceHub Token··········
Enter Your Google API Key··········


In [ ]:
from beyondllm import source,retrieve,embeddings,llms,generator

Load Data from YouTube Video

In [ ]:
data = source.fit(
    path="https://www.youtube.com/watch?v=ZM1bdh2mDJQ",
    dtype="youtube",
    chunk_size=1024,
    chunk_overlap=0)

In [ ]:
print(data)

[TextNode(id_='d8c9bfbf-7ff3-451c-814f-be0e2a41a7e4', embedding=None, metadata={'video_id': 'ZM1bdh2mDJQ'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='250c8287-9f69-43a3-98be-0e83bf052f96', node_type=<ObjectType.TEXT: '1'>, metadata={'video_id': 'ZM1bdh2mDJQ'}, hash='615e7d33180855d5371f6b37c8848fe6cb5ae545e1ba29af48aae860c75a6437')}, text="hi everyone welcome to the part seven\nvideo of building llm applications using\ngen stack in this video we are going to\nbuild a CSV agent using this tack so the\nCSV file will be the data source for the\nentire R pipeline so if you do not know\nwhat rag is then in short it's retrieval\naugmented generation it streamlines the\nprocess of retrieving data from a data\nsource then pre-processing it and\nstoring it in a certain way in a vector\nstore and then retri it when a query is\npassed and then presenting it with the\nhelp of a llm which we're going to\np

Set Embedding Model

In [ ]:
model_name='BAAI/bge-small-en-v1.5'

embed_model = embeddings.HuggingFaceEmbeddings(
    model_name=model_name
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Create Retriever

In [ ]:
retriever = retrieve.auto_retriever(
    data=data,
    embed_model=embed_model,
    type="hybrid",
    mode="OR",
    top_k=2)

In [ ]:
retrieved_nodes = retriever.retrieve("which tool is mentioned in the video?")
print(retrieved_nodes)

LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.
[NodeWithScore(node=TextNode(id_='d8c9bfbf-7ff3-451c-814f-be0e2a41a7e4', embedding=None, metadata={'video_id': 'ZM1bdh2mDJQ'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='250c8287-9f69-43a3-98be-0e83bf052f96', node_type=<ObjectType.TEXT: '1'>, metadata={'video_id': 'ZM1bdh2mDJQ'}, hash='615e7d33180855d5371f6b37c8848fe6cb5ae545e1ba29af48aae860c75a6437')}, text="hi everyone welcome to the part seven\nvideo of building llm applications using\ngen stack in this video we are going to\nbuild a CSV agent using this tack so the\nCSV file will be the data source for the\nentire R pipeline so if you do not know\nwhat rag is then in short it's retrieval\naugmented generation it streamlines the\nprocess of retrieving data from a data\nsource then pre-processing it and\nstoring it in a certain way in a v

Set Language Model

In [ ]:
llm = llms.GeminiModel(
    model_name="gemini-pro",
    google_api_key=google_api_key)

Generate Answer

In [ ]:
question = "what tool is the video mentioning about?"

In [ ]:
pipeline = generator.Generate(
    question=question,
    retriever=retriever,
    llm=llm)

LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.


In [ ]:
print(pipeline.call())

The video mentions the following tools:
- Gen Stack
- CSV loader
- Recursive character text splitter
- Hugging Face inference API embeddings
- Chroma Vector store
- Azure chat openi
- Combine docs chain
- Retrieval qm


In [ ]:
print(pipeline.get_rag_triad_evals())

Executing RAG Triad Evaluations...
Context relevancy Score: 1.0
This response does not meet the evaluation threshold. Consider refining the structure and content for better clarity and effectiveness.
Answer relevancy Score: 8.0
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.
Groundness score: 7.3
This response does not meet the evaluation threshold. Consider refining the structure and content for better clarity and effectiveness.
